# LDA Topic Modelling from Hansard
## Using LDA to try to extract topics from unlabelled UK parliamentary debates.

### Data preparation
A single-day example of the data can be found in `/data/2016-01-14.json` to make following this more clear.

In [1]:
import lzma
import pickle
from pprint import pprint

raw_data = None
with lzma.open(f"../data/2016-today.xz", "rb") as f:
    raw_data = pickle.load(f)

# Check loaded successfully by printing date of first 2 days in data.
pprint(raw_data[0]["date"])
pprint(raw_data[1]["date"])

'2016-01-05'
'2016-01-06'


Here we recursively loop through the "debates" as they have a nested structure. The problem we face here is that some things Hansard lists as "debates" are/have collections of debates or even just statements. So we try to filter down to the debates on the deepest level and those which have more than 1 contribution so therefore are not statements.

In [2]:
import numpy as np
import pandas as pd


def recurse_debates(debates):
    for deb in debates:  # Loop through debates
        contributions = [
            x for x in deb["items"] if x["type"] == "contribution"
        ]  # Get only the items which have contributions
        if (
            len(contributions) > 1
        ):  # Keep only debates that have more than one contribution.
            yield np.array(
                [deb["title"], " ".join([item["text"] for item in deb["items"]])]
            )  # Yield debate item in format [title, joined text]
        if (
            "child_debates" in deb
        ):  # If debate has child debates loop through those and yield items.
            for item in recurse_debates(deb["child_debates"]):
                yield item


def generate_debates(raw_data):
    for day in raw_data:  # Loop through days in data
        for item in recurse_debates(day["debates"]):
            yield item  # Yield each debate item


df = pd.DataFrame(list(generate_debates(raw_data)), columns=["title", "text"])
df

,title,text
0,Out-of-hospital Care,1. What progress his Department has made on i...
1,GP Services,2. What progress his Department is making on ...
2,Hospital Trusts: Deficits,3. What proportion of hospital trusts are in ...
3,Rare Diseases,4. How many people have diseases classified b...
4,Social Care Budgets: A&E Attendance,5. What assessment he has made of the effect ...
...,...,...
13988,Education Recovery,"With permission, Mr Deputy Speaker, I will mak..."
13989,Official Development Assistance,Application for emergency debate (Standing Ord...
13990,Points of Order,"On a point of order, Mr Deputy Speaker. This i..."
13991,Advanced Research and Invention Agency Bill,[Relevant documents: Third Report of the Scien...
